In [1]:
# ОПИСАНИЕ ЗАДАНИЯ
#Построй линейный прогноз любого из интересующего тебя пункта отчетности.
#Файлы которые я тебе скину - это реальная отчетность с интерфакса из под нашей OCR.
#Поэтому заодно применишь уже полученные знания в обработке плохо собранных таблиц
#Тебе нужно будет вытащить нужные ячейки, построить из этого график и спрогнозировать куда будет двигаться
#выбранный тобой пункт отчетности с последующими отчетностями.
#Как раз отработаешь уже изученные обработки таблиц и данных, построение графиков,
#методы группировки и скорее всего будешь вынуждена поработать с numpy, если не трогала его еще, как раз познакомишься.
#Ну и конечно задача регрессии машинного обучения что у тебя фактически актуальной темой является

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re

In [3]:
def add_balance(data):
    '''функция для добавления новых столбцов в df_balance'''
    data = data.dropna(how='all')
    for item in data.index:
        if data.iloc[item, 0] == 'Наименование показателя':
            i = item
            break
    new_column_list = data.iloc[i, 2:5].dropna().to_list()
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
    balance = data.iloc[(i+1):(j+1), 2:(2+len(new_column_list))]
    balance = balance.reset_index(drop=True)
    balance.columns = new_column_list
    return balance


def add_assets(data_new):
    '''функция для добавления активов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Внеоборотные активы Нематериальные активы', str(data_new.iloc[item, column])) != None:
                i = item
                c = column
                break
    new_column_list = data_new.iloc[(i-2), c:(c+4)].to_list()
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('сумма строк 1100', str(data_new.iloc[item, column])) != None:
                j = item
                break
    balance1 = data_new.iloc[i:(j), c:(c+4)]
    balance1 = balance1.reset_index(drop=True)
    balance1.columns = new_column_list
    balance1 = balance1.dropna()
    return balance1


def add_liabilities(data_new):
    '''функция для добавления пассивов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Капитал и резервы Уставный капитал', str(data_new.iloc[item, column])) != None:
                m = item
                n = column
                break
    new_column_list = data_new.iloc[(m-2), n:(n+4)].to_list()
    for item in data_new.index:
        if re.search('сумма строк 1300', str(data_new.iloc[item, n])) != None:
            j = item
            break
    balance2 = data_new.iloc[(m):(j+1), n:(n+4)]
    balance2 = balance2.reset_index(drop=True)
    balance2.columns = new_column_list
    balance2 = balance2.dropna()
    return balance2

In [4]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
# files_list = ['712_2509_0.xlsx', '713_2509_0.xlsx', '715_2509_0.xlsx', '716_2509_0.xlsx', '717_2509_0.xlsx', 
                #'719_2509_0.xlsx', '720_2509_0.xlsx', '721_2509_0.xlsx', '723_2509_0.xlsx', '724_2509_0.xlsx', 
                #'725_2509_0.xlsx', '727_2509_0.xlsx', '728_2509_0.xlsx', '729_2509_0.xlsx', '731_2509_0.xlsx', 
                #'732_2509_0.xlsx', '733_2509_0.xlsx', '735_2509_0.xlsx', '736_2509_0.xlsx', '737_2509_0.xlsx']

In [6]:
df_balance = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance = add_balance(data)
    df_balance[balance.columns] = balance

In [7]:
df_balance = df_balance.dropna(axis=1)

In [8]:
df_balance.columns = ['2020.06.30', '2019.12.31', '2020.03.31', '2019.09.30', '2017.12.31', '2018.06.30',
                      '2016.12.31', '2017.09.30', '2015.12.31', '2016.09.30',
                      '2015.12.31', '2014.12.31', '2016.06.30', '2016.03.31',
                      '2015.06.30', '2013.12.31', '2014.06.30', '2012.12.31']

In [9]:
df_balance = df_balance.loc[:, ~df_balance.columns.duplicated()]

In [10]:
df_balance = df_balance.reindex(sorted(df_balance.columns), axis=1)

In [11]:
data = pd.read_excel("./712_2509_0-Copy1.xlsx", header=None)
data = data.dropna(how='all')
for item in data.index:
    if data.iloc[item, 0] == 'Наименование показателя':
        i = item
        break
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
text_column_list = data.iloc[i, 0:2].dropna().to_list()
data = data.iloc[(i+1):(j+1), 0:2]
data = data.reset_index(drop=True)
data.columns = text_column_list
data[df_balance.columns] = df_balance

In [12]:
df_data = data.drop(0)

In [13]:
df_data.reset_index(drop=True, inplace=True)

In [14]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements1\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [27]:
# files_list ['722_2509_0.xlsx', '730_2509_0.xlsx', '734_2509_0.xlsx', '738_2509_0.xlsx']
# из 2-3 файлов не удалось совсем вытащить данные, но в итоге того, что есть хватило, т.к. в отчетность обучно включаются показатели
#прошлого и текущаго года. Отчетность 2010 и 2011 года вытаскивать пока не стала, там другой формат, нужно делать бухгалтерскую перекладку. Будет время - попробую.

In [16]:
df_balance1 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_assets(data)
    df_balance1[balance_new.columns] = balance_new

In [17]:
df_balance2 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_liabilities(data)
    df_balance2[balance_new.columns] = balance_new

In [18]:
balance_final = df_balance1.append(df_balance2, ignore_index=True)
balance_f = balance_final.drop(['Наименование показателя', 'Код'], axis=1)
balance_f.columns

Index(['На 31.12.2017', 'На 31.12.2016', 'На 31.12.2015', 'На 31.12.2014',
       'На 31.12.2013', 'На 31.12.2012'],
      dtype='object')

In [19]:
balance_f.columns = ['2017.12.31', '2016.12.31', '2015.12.31', '2014.12.31', '2013.12.31',
                     '2012.12.31']

In [20]:
data_final = pd.concat([df_data, balance_f], axis=1)
data_final = data_final.loc[:, ~data_final.columns.duplicated()]
data_final = data_final.reindex(sorted(data_final.columns), axis=1)

In [21]:
data_final = data_final.loc[:, ['Наименование показателя', 'Код', '2012.12.31', '2013.12.31', '2014.06.30',
                                '2014.12.31', '2015.06.30', '2015.12.31', '2016.03.31', '2016.06.30',
                                '2016.09.30', '2016.12.31', '2017.09.30', '2017.12.31', '2018.06.30',
                                '2019.09.30', '2019.12.31', '2020.03.31', '2020.06.30']]

In [22]:
for column in data_final.columns:
    data_final[column]= data_final[column].replace('—', 0) 
    data_final[column]= data_final[column].replace('-', 0) 

In [23]:
data_final = data_final.T
name_list = data_final.loc['Наименование показателя'].to_list()
data_final = data_final.drop('Наименование показателя')
data_final.columns = name_list
data_final.drop('Код', axis=0)
data_final = data_final.fillna(0)

In [24]:
data_final = data_final.drop('Код', axis=0)

In [25]:
data_final = data_final.replace(r'\s+','',regex=True)

In [26]:
data_final

,А К Т И В I. Внеоборотные активы Нематериальные активы,Результаты исследований и разработок,Основные средства,Финансовые вложения,Отложенные налоговые активы,Прочие внеоборотные активы,Итого по разделу I,II. Оборотные активы Запасы,Налог на добавленную стоимость по приобретенным ценностям,Дебиторская задолженность,...,Итого по разделу III,IV. Долгосрочные обязательства Заемные средства,Отложенные налоговые обязательства,Прочие обязательства,Итого по разделу IV,V. Краткосрочные обязательства Заемные средства,Кредиторская задолженность,Оценочные обязательства,Итого по разделу V,БАЛАНС (сумма строк 1300 + 1400 + 1500)
2012.12.31,1748963,28411,1032642,147780380,113130556,85019,861259,264667230,37661071,7934120,...,310161553,320609076,61455664,6762292,936305,69154261,44513482,30248292,1007390,75769164
2013.12.31,255884,37488,0,142980881,132979956,106873,3504849,279865931,37631821,8063796,...,293636474,304066597,90451782,8476574,158925,99087281,22336976,41868595,1744937,65950508
2014.06.30,316893,60360,0,139369815,138063122,115826,1909688,279835704,31396563,7681084,...,307292674,317709418,80503843,9069763,163687,89737293,32454324,37164647,1443632,71062603
2014.12.31,346166,30947,135334350,140632205,128932,1053135,277525735,37431161,8058219,111139719,...,314725971,96384209,9786365,409869,106580443,23556046,56053228,1442257,81051531,502357945
2015.06.30,441605,26633,132184057,151952921,138525,1426391,286170132,41147773,9235886,107465414,...,326732209,91974675,10843985,20902,102839562,22605714,63250178,2961852,88817744,518389515
2015.12.31,479344,28414,131024265,152520560,148597,1846963,286048143,42543057,7914804,123496442,...,328233790,137275933,11079929,22861,148378723,27893440,59011078,1911146,88815664,565428177
2016.03.31,669319,27154,132033566,145313809,150223,2038995,280233066,37710009,6392006,130057929,...,333939566,126276389,10879663,2846,137158898,30588262,48295229,3289188,82172679,553271143
2016.06.30,727720,25101,131564386,145564766,152158,2092533,280126664,39925142,6019980,61185524,...,330937283,128609326,10938742,2845,139550913,26011098,52955675,3383533,82350306,552838502
2016.09.30,720278,22216,130653226,145518903,151521,2068800,279134944,40407715,5526156,52840348,...,343403935,121838761,10889858,53030,132781649,18136223,54506043,3741293,76383559,552569143
2016.12.31,807061,22161,130676632,124336442,153987,1964808,257961091,53201748,6486168,134055175,...,315095233,95619746,11341530,12258,106973534,20796621,89819854,5144918,115761393,537830160
